# Configuração do Ambiente

In [ ]:
import pandas as pd
import os
from html import unescape # Ajsuta formatações HTML
from re import sub # Remove subpadrões
try:
  from unidecode import unidecode # Remove acentos
except ModuleNotFoundError:
  !pip install unidecode
  from unidecode import unidecode

# FORTES

In [ ]:
def clean_html(path: str) -> list[str]:
  """
  Lê um arquivo HTMl, remove todas as linhas que não tenham 'class'
  e remove todas as tags.

  Parâmetros:
  path: O caminho para o arquivo HTML a ser processado.

  Retorna uma lista de strings, sendo cada item uma linha do arquivo HTML.
  """
  # Lê o arquivo html e o separa por linhas
  with open(path, 'r') as file:
    arq = file.read()
  arq = arq.split('\n')

  # Filtra apenas as linhas que contenham 'class'
  # Para cada linha, ajusta a formatação HTML (unescape) e remove acentos (unidecode)
  arq = [unidecode(unescape(linha)) for linha in arq if 'class' in linha]
  # Remove as tags
  for i, row in enumerate(arq):
      pattern = r'<[^>]*>'
      new_row = sub(pattern, '', row)
      arq[i] = new_row

  return arq

In [ ]:
def FORTES(path: str) -> pd.DataFrame:

  # Tabela onde todos os dados serão guardados
  registros = pd.DataFrame(
      columns = ['DATA_FOLHA',	'EMPRESA',	'CNPJ', 'CODIGO_EMPREGADO',	
                 'NOME_EMPREGADO',	'CARGO',	'ADMISSAO',	'LOTACAO', 'CODIGO_EVENTO',
                 'NOME_EVENTO',	'TIPO',	'VALOR',	'CODIGO_EMPREGADO_FORM'])
  # Leitura do arquivo HTML
  arq = clean_html(path)

  # Lê alguns dados que são únicos para todo o arquivo
  competencia = arq[6]
  # Guarda apenas os números do CNPJ
  cnpj = ''.join([i for i in arq[8] if i.isnumeric()])
  empregador = arq[10]

  # O total de empregador na folha pode ser contado pela palavra chave 'Admissao',
  # que aparece no topo de cada contracheque
  tot_emp = arq.count('Admissao')

  start = 0
  for _ in range(tot_emp):

    # Encontra os dados do próximo empregado
    for i, row in enumerate(arq[start:], start=start):
      if row == 'Admissao':
        start = i + 1
        break
    # Acessa os dados do empregado pela posição das linhas
    admissao = arq[i+1]
    lotacao = arq[i+3]
    cargo = arq[i+5]
    """ 
    O código e o nome dos empregados estão na mesma linha, o formato 
    XXXXXX Nome Completo Empregado
    O código pode ser extraído com um row.split()[0]
    E o nome será o restante: row.split()[1:]
    O ' '.join() irá converter a lista para string.
    """
    cod_emp = arq[i+7].split()[0]
    nome_emp = ' '.join(arq[i+7].split()[1:])
    # É solicitada uma coluna com o código do empregado formatado com 6 dígitos
    cod_emp_form = f'{int(cod_emp):06}'
    
    # Procura o início dos eventos
    i += 8
    while arq[i] != 'Cod.':
      i += 1
    else: # Avança uma última linha para o início dos eventos
      i += 1
    
    eventos = list()
    evento = list()
    while arq[i] != 'Total de Proventos':
      evento.append(arq[i])
      if len(evento) == 5:
        eventos.append(evento)
        evento = list()
      i += 1
    # Descarta os eventos vazios verificando se há código de evento (último elemento)
    eventos = [evento for evento in eventos if evento[-1]]
    # Adiciona os eventos à tabela de registros
    for evento in eventos:
      # Verifica se é provento ou desconto
      if evento[0]:
        tipo = 'Desconto'
        valor = evento[0]
      else:
        tipo = 'Provento'
        valor = evento[1]
      cod_evento = evento[4]
      nome_evento = evento[3]
      registro = [competencia, empregador, cnpj, cod_emp, nome_emp, cargo, admissao,
                  lotacao, cod_evento, nome_evento, tipo, valor, cod_emp_form]
      registros.loc[len(registros)] = registro

  return registros

In [ ]:
FORTES('VIGON ADM 45 - CONTRACHEQUE.html')

# FIBRA

In [ ]:
def FIBRA(path: str) -> pd.DataFrame:

  folha = pd.read_excel(path).fillna('')
  # Guarda apenas as colunas necessárias
  folha.columns = ['Cod_Evento', 'Desc_Evento', 'Referência', 'Provento', 
                   'Desconto', '', 'Cod_Emp', 'Nome_Emp', '', 'Cargo', 
                   '', 'Competência', '', '']
  folha = folha[['Cod_Evento', 'Desc_Evento', 'Referência', 'Provento', 
                 'Desconto', 'Cod_Emp', 'Nome_Emp', 'Cargo', 'Competência']]
  
  # Tabela onde todos os dados serão guardados
  registros = pd.DataFrame(
      columns = ['DATA_FOLHA', 'EMPRESA', 'CNPJ', 'CODIGO_EMPREGADO',	
                 'NOME_EMPREGADO', 'CARGO', 'ADMISSAO', 'LOTACAO', 'CODIGO_EVENTO',
                 'NOME_EVENTO',	'TIPO',	'VALOR', 'CODIGO_EMPREGADO_FORM'])
  
  # Realiza a contagem de quantos funcionários estão na folha
  tot_func = folha['Cod_Evento'].to_list().count('Empregador')

  empregador = folha['Provento'].iloc[0]
  # Guarda apenas os números do CNPJ
  cnpj = folha['Desc_Evento'].iloc[0]
  cnpj = ''.join([i for i in cnpj if i.isnumeric()])
  # O mês está no formato de datetime, então precisa ser convertido
  meses = {
      '2024-01':'Janeiro de 2024',
      '2024-02':'Fevereiro de 2024',
      '2024-03':'Março de 2024',
      '2024-04':'Abril de 2024',
      '2024-05':'Maio de 2024',
      '2024-06':'Junho de 2024',
      '2024-07':'Julho de 2024',
      '2024-08':'Agosto de 2024',
      '2024-09':'Setembro de 2024',
      '2024-10':'Outubro de 2024',
      '2024-11':'Novembro de 2024',
      '2024-12':'Dezembro de 2024',
  }
  competencia = meses[str(folha['Competência'].iloc[1])[:7]]
  
  index = 0
  for _ in range(tot_func):
      while folha['Cod_Evento'].iloc[index] != 'Empregador':
        index += 1
      # Acessa as informações do empregado pela posição das linhas
      lotacao = folha['Referência'].iloc[index][6:]
      index += 1
      nome_emp = folha['Nome_Emp'].iloc[index]
      cod_emp = folha['Cod_Emp'].iloc[index]
      cod_emp_form = f'{int(cod_emp):06}'
      cargo = folha['Cargo'].iloc[index]
      admissao = ''
      # Início dos eventos
      index += 3
      start = index
      # Procura o fim dos eventos
      while folha['Cod_Evento'].iloc[index][:2].isnumeric():
        index += 1
      else:
        end = index
      # Guarda os eventos
      eventos = folha[['Cod_Evento', 'Desc_Evento', 'Provento', 'Desconto']].iloc[start:end]
      # Adiciona os eventos à tabela de registros
      for row in range(eventos.shape[0]):
        # Verifica se é provento ou desconto
        if eventos['Provento'].iloc[row]:
          tipo = 'Provento'
          valor = eventos['Provento'].iloc[row]
        else:
          tipo = 'Desconto'
          valor = eventos['Desconto'].iloc[row]
        cod_evento = eventos['Cod_Evento'].iloc[row]
        nome_evento = eventos['Desc_Evento'].iloc[row]
        registro = [competencia, empregador, cnpj, cod_emp, nome_emp, cargo, admissao,
                    lotacao, cod_evento, nome_evento, tipo, valor, cod_emp_form]
        registros.loc[len(registros)] = registro
        
  return registros

In [ ]:
FIBRA(os.listdir()[1])

# SOLUCAO

In [ ]:
def SOLUCAO(path: str) -> pd.DataFrame:

  # Leitura do arquivo
  arq = pd.read_excel(path).fillna('')

  # Tabela onde todos os dados serão guardados
  registros = pd.DataFrame(
      columns = ['DATA_FOLHA', 'EMPRESA', 'CNPJ', 'CODIGO_EMPREGADO',	
                 'NOME_EMPREGADO', 'CARGO', 'ADMISSAO', 'LOTACAO', 'CODIGO_EVENTO',
                 'NOME_EVENTO',	'TIPO',	'VALOR', 'CODIGO_EMPREGADO_FORM'])

  eventos = pd.DataFrame()

  competencia = str()
  empregador = str()
  nome_emp = str()
  admissao = str()
  cod_emp = str()
  lotacao = str()
  cargo = str()
  cnpj = str()

  index = 0
  start = 0
  end = int()
  proximo = bool()
  count_empregados = 0

  col_lotacao = str()
  col_descricoes = str()
  col_proventos = str()
  col_descontos = str()
  col_competencia = str()
  col_nome = str()

  # Definição das colunas que serão guardadas

  # Guarda o empregador que é um para o arquivo todo
  empregador = arq['Coluna3'].iloc[0]
  # Há um caso que o nome da empresa vem errado, a correção é feita abaixo.
  if empregador.strip() == 'MAIS SOLUCAO EM LOCACAO DE MAO DE OBRA I':
    empregador = 'MAIS SOLUCAO EM LOCACAO DE MAO DE OBRA INTEGRADA S.A.'
  # Encontra a coluna que contém o CNPJ
  for coluna in arq.columns[4:]:
    if arq[coluna].iloc[1]:
      cnpj = arq[coluna].iloc[1]
      cnpj = f'{int(cnpj):014}'
      break
  # Encontra a coluna em que está a lotação
  for coluna in arq:
    if arq[coluna].iloc[4] == 'Departamento':
      col_lotacao = coluna
      break
  # Encontra a coluna em que estão as descricoes
  for coluna in arq:
    if arq[coluna].iloc[8] == 'Descrição':
      col_descricoes = coluna
      break
  # Encontra a coluna em que estão os proventos
  for coluna in arq:
    if arq[coluna].iloc[8] == 'Vencimentos':
      col_proventos = coluna
      break
  # Encontra a coluna em que estão os descontos
  for coluna in arq:
    if arq[coluna].iloc[8] == 'Descontos':
      col_descontos = coluna
      break
  # Encontra a coluna em que estão as competências
  for i, coluna in enumerate(arq):
    if arq[coluna].iloc[1] == 'Folha Mensal':
      col_competencia = arq.columns[i+1]
      break
  # Encontra a coluna em que está o nome
  for coluna in arq:
    if arq[coluna].iloc[4] == 'Nome do Funcionário':
      col_nome = coluna
      break
  # Encontra a coluna em que está a admissão
  # Neste caso é preciso de um indicador booleano pois na linha 6 primeiro
  # aparece 'Admissão' e depois a admissão, então é preciso ignorar a primeira coluna.
  proximo = False
  for coluna in arq:
    if arq[coluna].iloc[6]:
      if proximo:
        col_admissao = coluna
        break
      elif arq[coluna].iloc[6] == 'Admissão:':
        proximo = True

  # Guarda apenas as colunas necessárias e as renomeia
  colunas = ['Coluna2', col_descricoes, col_proventos, col_descontos, col_nome, col_admissao, col_lotacao, col_competencia]
  arq = arq[colunas]
  arq.columns = ['Cod_Evento', 'Desc_Evento', 'Provento', 'Desconto', 'Nome_Empregado', 'Admissão', 'Lotação', 'Competência']

  # Conta o total de empregados na folha

  for i in arq['Cod_Evento'].tolist():
    if i == 'Código':
      count_empregados += 1

  # Início da coleta de dados

  for _ in range(count_empregados):

    # Encontra a palavra chave 'Código' que será chave para achar os outros dados
    # e indica o início dos dados de um funcionário.
    start = index
    # Encontra os dados do próximo funcionário
    for i, row in enumerate(arq['Cod_Evento'].tolist()[start:], start=start):
      if row == 'Código':
        index = i+1
        break
    # Coleta os outros dados que estão 'ao redor' da palavra 'Código'
    # Verifica se o empregado atual é o mesmo da folha anterior
    novo_cod = arq['Cod_Evento'].iloc[index-4]
    if cod_emp == novo_cod:
      continue
    cod_emp = novo_cod
    cod_emp_form = f'{int(cod_emp):06}'
    nome_emp = arq['Nome_Empregado'].iloc[index-4]
    cargo = arq['Nome_Empregado'].iloc[index-3]
    admissao = str(arq['Admissão'].iloc[index-3])
    lotacao = arq['Lotação'].iloc[index-4]
    competencia = arq['Competência'].iloc[index-7]

    admissao = f'{admissao[8:10]}/{admissao[5:7]}/{admissao[:4]}'
    # Coleta os eventos

    end = index
    # Procura o fim dos eventos
    while arq['Cod_Evento'].iloc[end]:
      end += 1

    eventos = arq[['Cod_Evento', 'Desc_Evento', 'Provento', 'Desconto']].iloc[index:end]

    # Adiciona os eventos à tabela de registros
    for row in range(eventos.shape[0]):
      # Verifica se é provaneto ou desconto
      if eventos['Provento'].iloc[row]:
        tipo = 'Provento'
        valor = eventos['Provento'].iloc[row]
      else:
        tipo = 'Desconto'
        valor = eventos['Desconto'].iloc[row]

      cod_evento = eventos['Cod_Evento'].iloc[row]
      nome_evento = eventos['Desc_Evento'].iloc[row]
      registro = [competencia, empregador, cnpj, cod_emp, nome_emp, cargo, admissao,
                  lotacao, cod_evento, nome_evento, tipo, valor, cod_emp_form]
      registros.loc[len(registros)] = registro


  return registros

In [ ]:
SOLUCAO(os.listdir()[1])

# Junção de planilhas

In [ ]:
folha = pd.DataFrame()
files = [file for file in os.listdir() if '.xlsx' in file]
for file in files:
  folha_temp = pd.read_excel(file)
  folha = pd.concat([folha, folha_temp])

# Ajusta o CNPJ que volta para o tipo int
folha['CNPJ'] = folha['CNPJ'].apply(lambda x: f'{int(x):014}')
# Corrige lotações com espaços no final
folha['LOTACAO'] = folha['LOTACAO'].apply(lambda x: x.strip())
# Remove acentos das colunas
for col in ['EMPRESA', 'NOME_EMPREGADO', 'LOTACAO', 'NOME_EVENTO']:
  folha[col] = folha[col].apply(str)
  folha[col] = folha[col].apply(unidecode)
# Ajusta lotações que começam com números
folha['LOTACAO'] = folha['LOTACAO'].apply(
                       lambda x: x[x.find(' - ')+3:] if x[0].isnumeric() else x)

folha.to_excel('Folha GERAL.xlsx', index=False)